# Finding Best LSTM-CNN Regressor for Store Item Demand Forecasting

## Import required libraries

In [ ]:
!pip install keras-tuner 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
pd.options.display.max_columns = 12
pd.options.display.max_rows = 24

import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()

import numpy as np

%config InlineBackend.figure_format = 'svg'

from pylab import rcParams
rcParams['figure.figsize'] = 5, 4

from itertools import product, starmap

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model
from keras.layers import *

import time
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.losses import MeanSquaredError

from keras import models 

from scipy.stats import describe
from sklearn.metrics import mean_squared_error, r2_score

import kerastuner as kt

## Read datasets

In [ ]:
# google colab
from google.colab import drive
drive.mount('/content/drive')

df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml project/train.csv', parse_dates = ['date'])
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml project/test.csv', parse_dates = ['date'])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # jupyter
# df_train = pd.read_csv('train.csv', parse_dates = ['date'])
# df_test = pd.read_csv('test.csv', parse_dates = ['date'])

In [ ]:
df_train.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [ ]:
df_test.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


## Preprocess data

### Convert strings to dates

In [ ]:
df_train['date'] = pd.to_datetime(df_train['date'])
df_train.index = pd.DatetimeIndex(df_train['date'])
df_train.drop('date', axis=1, inplace=True)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 913000 entries, 2013-01-01 to 2017-12-31
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   store   913000 non-null  int64
 1   item    913000 non-null  int64
 2   sales   913000 non-null  int64
dtypes: int64(3)
memory usage: 27.9 MB


In [ ]:
df_test['date'] = pd.to_datetime(df_test['date'])
df_test.index = pd.DatetimeIndex(df_test['date'])
df_test.drop('date', axis=1, inplace=True)

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 45000 entries, 2018-01-01 to 2018-03-31
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   id      45000 non-null  int64
 1   store   45000 non-null  int64
 2   item    45000 non-null  int64
dtypes: int64(3)
memory usage: 1.4 MB


### Sales for each storeitem

In [ ]:
def storeitems():
    return product(range(1,51), range(1,11))


def storeitems_column_names():
    return list(starmap(lambda i,s: f'item_{i}_store_{s}_sales', storeitems()))


def sales_by_storeitem(df):
    ret = pd.DataFrame(index=df.index.unique())
    for i, s in storeitems():
        ret[f'item_{i}_store_{s}_sales'] = df[(df['item'] == i) & (df['store'] == s)]['sales'].values
    return ret

In [ ]:
df_train = sales_by_storeitem(df_train)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1826 entries, 2013-01-01 to 2017-12-31
Columns: 500 entries, item_1_store_1_sales to item_50_store_10_sales
dtypes: int64(500)
memory usage: 7.0 MB


### Mock sales to use same transformations as train data

In [ ]:
df_test['sales'] = np.zeros(df_test.shape[0])
df_test = sales_by_storeitem(df_test)

In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90 entries, 2018-01-01 to 2018-03-31
Columns: 500 entries, item_1_store_1_sales to item_50_store_10_sales
dtypes: float64(500)
memory usage: 352.3 KB


### Combine train and test data

In [ ]:
# ensure columns have same name and order
col_names = list(zip(df_test.columns, df_train.columns))
for cn in col_names:
    assert cn[0] == cn[1]

In [ ]:
df_test['is_test'] = np.repeat(True, df_test.shape[0])
df_train['is_test'] = np.repeat(False, df_train.shape[0])
df_total = pd.concat([df_train, df_test])
df_total.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1916 entries, 2013-01-01 to 2018-03-31
Columns: 501 entries, item_1_store_1_sales to is_test
dtypes: bool(1), float64(500)
memory usage: 7.3 MB


### One-Hot encoding

In [ ]:
weekday_df = pd.get_dummies(df_total.index.weekday, prefix='weekday')
weekday_df.index = df_total.index

weekday_df.head()

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6
date,,,,,,,
2013-01-01,0,1,0,0,0,0,0
2013-01-02,0,0,1,0,0,0,0
2013-01-03,0,0,0,1,0,0,0
2013-01-04,0,0,0,0,1,0,0
2013-01-05,0,0,0,0,0,1,0


In [ ]:
month_df = pd.get_dummies(df_total.index.month, prefix='month')
month_df.index =  df_total.index

month_df.head()

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
date,,,,,,,,,,,,
2013-01-01,1,0,0,0,0,0,0,0,0,0,0,0
2013-01-02,1,0,0,0,0,0,0,0,0,0,0,0
2013-01-03,1,0,0,0,0,0,0,0,0,0,0,0
2013-01-04,1,0,0,0,0,0,0,0,0,0,0,0
2013-01-05,1,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
weekyear_df = pd.get_dummies(df_total.index.weekofyear, prefix='weekyear')
weekyear_df.index =  df_total.index

weekyear_df.head()

,weekyear_1,weekyear_2,weekyear_3,weekyear_4,weekyear_5,weekyear_6,...,weekyear_48,weekyear_49,weekyear_50,weekyear_51,weekyear_52,weekyear_53
date,,,,,,,,,,,,,
2013-01-01,1,0,0,0,0,0,...,0,0,0,0,0,0
2013-01-02,1,0,0,0,0,0,...,0,0,0,0,0,0
2013-01-03,1,0,0,0,0,0,...,0,0,0,0,0,0
2013-01-04,1,0,0,0,0,0,...,0,0,0,0,0,0
2013-01-05,1,0,0,0,0,0,...,0,0,0,0,0,0


In [ ]:
quarter_df = pd.get_dummies(df_total.index.quarter, prefix='quarter')
quarter_df.index =  df_total.index

quarter_df.head()

,quarter_1,quarter_2,quarter_3,quarter_4
date,,,,
2013-01-01,1,0,0,0
2013-01-02,1,0,0,0
2013-01-03,1,0,0,0
2013-01-04,1,0,0,0
2013-01-05,1,0,0,0


In [ ]:
df_total = pd.concat([weekday_df, month_df, weekyear_df, quarter_df, df_total], axis=1)

df_total.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1916 entries, 2013-01-01 to 2018-03-31
Columns: 577 entries, weekday_0 to is_test
dtypes: bool(1), float64(500), uint8(76)
memory usage: 7.5 MB


In [ ]:
assert df_total.isna().any().any() == False

### Shift sales 


In [ ]:
def shift_series(series, days):
    return series.transform(lambda x: x.shift(days))


def shift_series_in_df(df, series_names=[], days_delta=90):
    """
    Shift columns in df with names in series_names by days_delta.
    
    Negative days_delta will prepend future values to current date,
    positive days_delta wil prepend past values to current date.
    """
    ret = pd.DataFrame(index=df.index.copy())
    str_sgn = 'future' if np.sign(days_delta) < 0 else 'past'
    for sn in series_names:
        ret[f'{sn}_{str_sgn}_{np.abs(days_delta)}'] = shift_series(df[sn], days_delta)
    return ret

    
def stack_shifted_sales(df, days_delta=90):
    names = storeitems_column_names()
    dfs = [df.copy()]
    abs_range = range(1, days_delta+1) if days_delta > 0 else range(days_delta, 0)
    for day_offset in abs_range:
        delta = -day_offset
        shifted = shift_series_in_df(df, series_names=names, days_delta=delta)
        dfs.append(shifted)
    return pd.concat(dfs, axis=1, copy=False)

In [ ]:
df_total = stack_shifted_sales(df_total, days_delta=-1)

In [ ]:
# remove 1st row
df_total = df_total.dropna()  

df_total.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1915 entries, 2013-01-02 to 2018-03-31
Columns: 1077 entries, weekday_0 to item_50_store_10_sales_past_1
dtypes: bool(1), float64(1000), uint8(76)
memory usage: 14.8 MB


In [ ]:
# ensure stacked and standard sales columns in same order
sales_cols = [col for col in df_total.columns if '_sales' in col and '_sales_' not in col]
stacked_sales_cols = [col for col in df_total.columns if '_sales_' in col]
other_cols = [col for col in df_total.columns if col not in set(sales_cols) and col not in set(stacked_sales_cols)]

sales_cols = sorted(sales_cols)
stacked_sales_cols = sorted(stacked_sales_cols)

new_cols = other_cols + stacked_sales_cols + sales_cols

In [ ]:
df_total = df_total.reindex(columns=new_cols)

In [ ]:
df_total.head()

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,...,item_9_store_4_sales,item_9_store_5_sales,item_9_store_6_sales,item_9_store_7_sales,item_9_store_8_sales,item_9_store_9_sales
date,,,,,,,,,,,,,
2013-01-02,0,0,1,0,0,0,...,21.0,20.0,17.0,20.0,28.0,36.0
2013-01-03,0,0,0,1,0,0,...,25.0,15.0,28.0,18.0,31.0,25.0
2013-01-04,0,0,0,0,1,0,...,37.0,20.0,33.0,24.0,46.0,31.0
2013-01-05,0,0,0,0,0,1,...,37.0,23.0,27.0,14.0,35.0,30.0
2013-01-06,0,0,0,0,0,0,...,37.0,29.0,20.0,24.0,34.0,35.0


In [ ]:
df_total.describe()

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,...,item_9_store_4_sales,item_9_store_5_sales,item_9_store_6_sales,item_9_store_7_sales,item_9_store_8_sales,item_9_store_9_sales
count,1915.000000,1915.000000,1915.000000,1915.000000,1915.000000,1915.000000,...,1915.000000,1915.000000,1915.000000,1915.000000,1915.000000,1915.000000
mean,0.142559,0.142559,0.143081,0.143081,0.143081,0.143081,...,51.522193,37.021410,37.613055,34.219321,60.241253,51.577023
std,0.349714,0.349714,0.350247,0.350247,0.350247,0.350247,...,18.707446,13.864136,13.968996,12.788732,21.714863,18.706826
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,41.000000,29.000000,30.000000,27.000000,48.000000,41.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,52.000000,38.000000,38.000000,35.000000,61.000000,52.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,64.000000,46.000000,46.000000,42.000000,74.000000,63.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,111.000000,84.000000,81.000000,78.000000,134.000000,110.000000


In [ ]:
assert df_total.isna().any().any() == False

### Scaling

In [ ]:
cols_to_scale = [col for col in df_total.columns if 'weekday' not in col and 'month' not in col and 'weekyear' not in col and 'quarter' not in col]

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_cols = scaler.fit_transform(df_total[cols_to_scale])
df_total[cols_to_scale] = scaled_cols

df_total.head()

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,...,item_9_store_4_sales,item_9_store_5_sales,item_9_store_6_sales,item_9_store_7_sales,item_9_store_8_sales,item_9_store_9_sales
date,,,,,,,,,,,,,
2013-01-02,0,0,1,0,0,0,...,0.189189,0.238095,0.209877,0.256410,0.208955,0.327273
2013-01-03,0,0,0,1,0,0,...,0.225225,0.178571,0.345679,0.230769,0.231343,0.227273
2013-01-04,0,0,0,0,1,0,...,0.333333,0.238095,0.407407,0.307692,0.343284,0.281818
2013-01-05,0,0,0,0,0,1,...,0.333333,0.273810,0.333333,0.179487,0.261194,0.272727
2013-01-06,0,0,0,0,0,0,...,0.333333,0.345238,0.246914,0.307692,0.253731,0.318182


In [ ]:
df_total.describe()

,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,...,item_9_store_4_sales,item_9_store_5_sales,item_9_store_6_sales,item_9_store_7_sales,item_9_store_8_sales,item_9_store_9_sales
count,1915.000000,1915.000000,1915.000000,1915.000000,1915.000000,1915.000000,...,1915.000000,1915.000000,1915.000000,1915.000000,1915.000000,1915.000000
mean,0.142559,0.142559,0.143081,0.143081,0.143081,0.143081,...,0.464164,0.440731,0.464359,0.438709,0.449562,0.468882
std,0.349714,0.349714,0.350247,0.350247,0.350247,0.350247,...,0.168536,0.165049,0.172457,0.163958,0.162051,0.170062
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.369369,0.345238,0.370370,0.346154,0.358209,0.372727
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.468468,0.452381,0.469136,0.448718,0.455224,0.472727
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.576577,0.547619,0.567901,0.538462,0.552239,0.572727
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Split back to train and test data

In [ ]:
df_train = df_total[df_total['is_test'] == False].drop('is_test', axis=1)
df_test = df_total[df_total['is_test'] == True].drop('is_test', axis=1)

In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1825 entries, 2013-01-02 to 2017-12-31
Columns: 1076 entries, weekday_0 to item_9_store_9_sales
dtypes: float64(1000), uint8(76)
memory usage: 14.1 MB


In [ ]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 90 entries, 2018-01-01 to 2018-03-31
Columns: 1076 entries, weekday_0 to item_9_store_9_sales
dtypes: float64(1000), uint8(76)
memory usage: 710.5 KB


### Prepare training data

In [ ]:
X_cols_stacked = [col for col in df_train.columns if '_past_' in col]
X_cols_caldata = [col for col in df_train.columns if 'weekday_' in col or 'month_' in col or 'weekyear_' in col or 'quarter_' in col]
X_cols = X_cols_stacked + X_cols_caldata

X = df_train[X_cols]

In [ ]:
X_colset = set(X_cols)
y_cols = [col for col in df_train.columns if col not in X_colset]

y = df_train[y_cols]

In [ ]:
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1825 entries, 2013-01-02 to 2017-12-31
Columns: 576 entries, item_10_store_10_sales_past_1 to quarter_4
dtypes: float64(500), uint8(76)
memory usage: 7.1 MB


In [ ]:
y.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1825 entries, 2013-01-02 to 2017-12-31
Columns: 500 entries, item_10_store_10_sales to item_9_store_9_sales
dtypes: float64(500)
memory usage: 7.0 MB


In [ ]:
# split training data into train and val (80-20)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, shuffle=False)

In [ ]:
X_train.shape, X_valid.shape

((1460, 576), (365, 576))

In [ ]:
y_train.shape, y_valid.shape

((1460, 500), (365, 500))

In [ ]:
# reshape inputs to be 3d
X_train_vals = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_valid_vals = X_valid.values.reshape((X_valid.shape[0], 1, X_valid.shape[1]))

In [ ]:
X_train_vals.shape, X_valid_vals.shape

((1460, 1, 576), (365, 1, 576))

## LSTM-CNN hyperparameter tuning

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [ ]:
def build_model(hp):
    inputs = Input(shape=(X_train_vals.shape[1], X_train_vals.shape[2]))
    
    # top pipeline
    top_lstm = LSTM(500, return_sequences=True)(inputs)
    top_dense = Dense(500, activation='relu')(top_lstm)
    
    # bottom pipeline
    bottom_dense = Dense(500)(inputs)
    bottom_conv1 = Conv1D(
        hp.Int('1-neuron', min_value=600, max_value=800, step=50), 
        kernel_size=1,
        input_shape=(X_train_vals.shape[1], X_train_vals.shape[2])
    )(bottom_dense)
    bottom_conv2 = Conv1D(
        hp.Int('2-neuron', min_value=600, max_value=800, step=50),
        kernel_size=50,
        padding='same',
        activation='relu'
    )(bottom_conv1)
    bottom_conv3 = Conv1D(
        hp.Int('3-neuron', min_value=500, max_value=700, step=50),
        kernel_size=10,
        padding='same',
        activation='relu'
    )(bottom_conv2)
    bottom_pooling = AvgPool1D(
        pool_size=10, 
        padding='same'
    )(bottom_conv3)

    # concat output
    final_concat = Concatenate()([top_dense, bottom_pooling])
    final_lstm = LSTM(1000, dropout=(hp.Float('4-dropout', min_value=0.0, max_value=0.4, step=0.1)))(final_concat)
    final_dense = Dense(500)(final_lstm)
    
    # compile and return
    model = Model(inputs=inputs, outputs=final_dense)
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    #model.summary()
    
    return model

In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='mse',
    max_trials=10,
    project_name='lstm-cnn_tune-results')

INFO:tensorflow:Reloading Oracle from existing project .\lstm-cnn_tune1-results\oracle.json
INFO:tensorflow:Reloading Tuner from .\lstm-cnn_tune1-results\tuner0.json


In [ ]:
tuner.search(x=X_train_vals, 
             y=y_train.values, 
             validation_data=(X_valid_vals, y_valid.values),
             epochs=100,
             batch_size=128,
             callbacks=[early_stop])

INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.results_summary()

In [ ]:
tuner.results_summary()

Results summary
Results in .\lstm-cnn_tune1-results
Showing 10 best trials
Trial summary
Hyperparameters:
1-neuron: 650
2-neuron: 700
3-neuron: 600
4-dropout: 0.2
Score: 0.005532809998840094
Trial summary
Hyperparameters:
1-neuron: 750
2-neuron: 700
3-neuron: 650
4-dropout: 0.30000000000000004
Score: 0.005566848441958427
Trial summary
Hyperparameters:
1-neuron: 650
2-neuron: 800
3-neuron: 650
4-dropout: 0.2
Score: 0.005598306190222502
Trial summary
Hyperparameters:
1-neuron: 750
2-neuron: 750
3-neuron: 650
4-dropout: 0.30000000000000004
Score: 0.005618677940219641
Trial summary
Hyperparameters:
1-neuron: 600
2-neuron: 700
3-neuron: 700
4-dropout: 0.30000000000000004
Score: 0.005641312338411808
Trial summary
Hyperparameters:
1-neuron: 750
2-neuron: 700
3-neuron: 550
4-dropout: 0.1
Score: 0.0056538707576692104
Trial summary
Hyperparameters:
1-neuron: 650
2-neuron: 650
3-neuron: 550
4-dropout: 0.1
Score: 0.005666086915880442
Trial summary
Hyperparameters:
1-neuron: 750
2-neuron: 750
3-neu

In [ ]:
print('1: ', tuner.get_best_hyperparameters()[0].get('1-neuron'))
print('2: ', tuner.get_best_hyperparameters()[0].get('2-neuron'))
print('3: ', tuner.get_best_hyperparameters()[0].get('3-neuron'))
print('4: ', tuner.get_best_hyperparameters()[0].get('4-dropout'))

1:  650
2:  700
3:  600
4:  0.2
